There is a change in preprocessing in this notebook, we removed all the numbers from tweets, which helped in training more robust word2vec embeddings

Analysis on scraped dataset

### Import

In [0]:
import pandas as pd
import numpy as np

In [0]:
import pickle
import sys
import nltk
from nltk.stem.porter import *

from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import seaborn
%matplotlib inline

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
cf_data_1 = pd.read_csv('/content/drive/My Drive/Hate_Speech_Detection_git/data_1/hatespeech_NAACL_SRW.csv',encoding = "ISO-8859-1")
cf_data_2 = pd.read_csv('/content/drive/My Drive/Hate_Speech_Detection_git/data_1/hatespeech_NLP+CSS.csv')

cf_data_3 = pd.read_csv('/content/drive/My Drive/Hate_Speech_Detection_git/data_2/labeled_data.csv',encoding = "ISO-8859-1")
## this is the scraped data

cf_data_3.rename({'Unnamed: 0':'ID','tweet':'Tweets'},axis=1,inplace=True)

labels_1 = pd.read_csv('/content/drive/My Drive/Hate_Speech_Detection_git/data_1/NAACL_SRW_2016.csv',header=None,names=['ID','class'])
labels_2 = pd.read_csv('/content/drive/My Drive/Hate_Speech_Detection_git/data_1/NLP+CSS_2016.csv',sep='\s')

labels_2.rename({'TweetID':'ID','Expert':'class'},axis=1,inplace=True)

cf_data_1.rename({'Unnamed: 0':'index_col'},axis=1,inplace=True)
cf_data_2.rename({'Unnamed: 0':'index_col'},axis=1,inplace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  # Remove the CWD from sys.path while we load stuff.


In [0]:
real_world_test = pd.read_csv('/content/drive/My Drive/Hate_Speech_Detection_git/real_world_test/realDonaldTrump_tweets_extended.csv')

In [0]:
real_world_test_racist = pd.read_csv('/content/drive/My Drive/Hate_Speech_Detection_git/real_world_test/racist_scarped_extended.csv', names=['timestamp','text'])

In [0]:
pok = pd.read_csv('/content/drive/My Drive/Hate_Speech_Detection_git/real_world_test/pok.csv', names=['timestamp','text'])

In [0]:
pok = pok.drop_duplicates(subset='text')

### Function for merging

In [0]:
def label_merging(data, labels):
    labels['ID'] = labels['ID'].astype(int)
    print(labels['ID'].nunique())
    print('Null IDs in data 1 = ' ,data['ID'].isna().sum())
    
    data['ID'].fillna(0,inplace=True)
    data['ID'] = data['ID'].astype(int)
    
    print('data shape ='  ,data.shape)
    print('IDs common in data and labels =',sum(data['ID'].isin(labels['ID'])))
    
    train = data.merge(labels, on='ID',how='inner')#['class'].isna().sum()
    return train

In [10]:
train_1 = label_merging(cf_data_1,labels_1)

train_2 = label_merging(cf_data_2, labels_2)

train_3 = cf_data_3.copy()

16849
Null IDs in data 1 =  2
data shape = (16037, 11)
IDs common in data and labels = 11238
6909
Null IDs in data 1 =  0
data shape = (6271, 11)
IDs common in data and labels = 6271


In [0]:
t1 = train_1[['ID','Tweets','class']]
t2 = train_2[['ID','Tweets','class']]
t3 = train_3[['ID','Tweets','class']]
merged = pd.concat([t1,t2,t3],axis=0).reset_index(drop=True)

### Basic Preprocessing

In [0]:
train = merged.copy()

train.rename(columns={'Tweets':'tweet'},inplace=True)
train['tweet'] = train['tweet'].astype(str)

In [0]:
def clean_remove_b(data):  
  data.rename(columns={'text':'tweet'},inplace=True)
  data['tweet'] = data['tweet'].astype(str)

  data['tweet'] = data['tweet'].apply(lambda x:x[2:] if x[0:2]=="b'" or 'b"' else x)

In [0]:
clean_remove_b(real_world_test)
clean_remove_b(real_world_test_racist)
clean_remove_b(pok)

Every word followed by @ is some twitter ID of an user, which shouldn't be considered in our analysis, so lets do the stemming, where we remove @ alonwith the word followed by it

In [0]:
def preprocess(data):
  data['tweet'] = data['tweet'].apply(lambda x:' '.join(i for i in [a for a in x.split() if a.find('@')==-1]))
  data['tweet'] = data['tweet'].apply(lambda x:' '.join(i for i in [a for a in x.split() if a.find('http')==-1]))
  
  ## we are removing hashtags now, but while doing transfer learning, to learn the embeddings we didnt remove these, 
  ## just to include such words in our vocabulary
  
  data['tweet'] = data['tweet'].apply(lambda x:' '.join(i for i in [a for a in x.split() if a.find('#')==-1]))
  data['tweet'] = data['tweet'].apply(lambda x:''.join([i for i in x if not i.isdigit()]))
  data['tweet'] = data['tweet'].apply(lambda x: " ".join(x.lower() for x in x.split()))
  data['tweet'] = data['tweet'].str.replace('[^\w\s]','')

  import nltk
  nltk.download('stopwords')

  from nltk.corpus import stopwords
  stop = stopwords.words('english')
  data['tweet'] = data['tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

  remove_word = ['rt','mkr','im']
  data['tweet'] = data['tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in remove_word))

In [16]:
preprocess(train)
preprocess(real_world_test)
preprocess(real_world_test_racist)
preprocess(pok)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Doesnt really make sense to remove rare words, i.e. the words with count 1. Because we might lose hateful words this way

In [0]:
def preprocess_2(data):
  from textblob import TextBlob
  nltk.download('punkt')

  from textblob import Word
  nltk.download('wordnet')
  data['tweet'] = data['tweet'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
  data['tweet'].head()

In [18]:
preprocess_2(train)
preprocess_2(real_world_test)
preprocess_2(real_world_test_racist)
preprocess_2(pok)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:
# real_world_test = real_world_test[real_world_test['tweet'].apply(lambda x:len(x)>1)]

### Target creation

In [20]:
train['class'].unique()#.isna().sum()

array(['racism', 'none', 'sexism', 'neither', 'both', 2, 1, 0],
      dtype=object)

In [21]:
train['class'].replace(['racism', 'sexism',0, 1, 'both', 'none', 'neither',2],['hate','hate','hate','hate','hate','null','null','null'],inplace=True)
train['class'].value_counts()

hate    24942
null    17422
Name: class, dtype: int64

In [0]:
train['class'].replace(['null','hate'],[0,1],inplace=True)

### Creating text files for CNN training

In [0]:
# hate_text = train[train['class']==1]['tweet']
# null_text = train[train['class']==0]['tweet']

In [0]:
# hate_text.to_csv(r'hate_speech.txt', header=None, index=None, sep=' ')

In [0]:
# null_text.to_csv(r'null_speech.txt', header=None, index=None, sep=' ')

In [26]:
sum(train['ID'].value_counts()>1)

2760

In [27]:
train.shape

(42364, 3)

In [0]:
train = train.drop_duplicates(subset='ID')

real_world_test = real_world_test.drop_duplicates(subset='id')

### Basic Model on whole dataset

In [0]:
from sklearn.model_selection import train_test_split
x_train,x_valid,y_t,y_v = train_test_split(train['tweet'],train['class'],test_size=0.2,random_state=234)

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_features=100, lowercase=True, analyzer='word',
 stop_words= 'english',ngram_range=(1,1))

tfidf.fit(train['tweet'])

x_t = tfidf.transform(x_train)
x_v = tfidf.transform(x_valid)


In [0]:
def model_training(clf, x_t, y_t, x_v=None , y_v=None ,task='binary:logistic'):
    clf.fit(x_t,y_t)
    print('training accuracy', clf.score(x_t,y_t))
    
    if task=='binary:logistic':
      print('validation accuracy', clf.score(x_v,y_v))
      print('validation f1_score',f1_score(clf.predict(x_v),y_v))
      print('validation roc_auc score',roc_auc_score(y_v,clf.predict_proba(x_v)[::,-1]))
      print('confusion matrix \n',confusion_matrix(y_v, clf.predict(x_v)))
    
    if task=='reg:linear':
        if x_v!=None:
            print('validation r2_score', clf.score(x_v,y_v))
            print('validation MSE',mean_squared_error(clf.predict(x_v),y_v))

            
    return clf

In [32]:
# y_t.value_counts()

19794/11810

1.6760372565622355

In [33]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import f1_score, classification_report, roc_auc_score, confusion_matrix, accuracy_score
from xgboost import XGBClassifier

# lgr =  LogisticRegression(n_jobs=1, C=1e5)
xgb = XGBClassifier(n_estimators=500, max_depth=5,learning_rate=0.1,scale_pos_weight=1.4266790777602751)
# xgb = XGBClassifier(n_estimators=500, max_depth=5,learning_rate=0.1,scale_pos_weight=1.6760372565622355)
model_training(xgb,x_t,y_t,x_v,y_v)


training accuracy 0.8697316795342361
validation accuracy 0.8521893191597064
validation f1_score 0.8771819137749738
validation roc_auc score 0.915704695718503
confusion matrix 
 [[2563  334]
 [ 834 4171]]


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=None, n_estimators=500, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1.4266790777602751,
              seed=None, silent=None, subsample=1, verbosity=1)

In [0]:
# training accuracy 0.8751740286039742
# validation accuracy 0.8567451278157429
# validation f1_score 0.8815403934700711
# validation roc_auc score 0.9185450724629186
# [[2558  339]
#  [ 793 4212]]

### Training on new word2vec model

In [35]:
import gensim
import logging
from gensim.models import Word2Vec

wv = gensim.models.KeyedVectors.load_word2vec_format("/content/drive/My Drive/Hate_Speech_Detection_git/model_transfer_learning.txt", binary=False)
wv.init_sims(replace=True)


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
def word_averaging(wv, words):
    all_words, mean = set(), []
    
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in wv.vocab:
            mean.append(wv.syn0norm[wv.vocab[word].index])
            all_words.add(wv.vocab[word].index)

    if not mean:
        logging.warning("cannot compute similarity with no input %s", words)
        # FIXME: remove these examples in pre-processing
        return np.zeros(wv.vector_size,)

    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

  
def  word_averaging_list(wv, text_list):
    return np.vstack([word_averaging(wv, post) for post in text_list ])

In [37]:
from sklearn.model_selection import train_test_split

def w2v_tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text, language='english'):
        for word in nltk.word_tokenize(sent, language='english'):
            if len(word) < 2:
                continue
            tokens.append(word)
    return tokens
    
train_w2v, test_w2v = train_test_split(train, test_size=0.2, random_state = 42)
# x_t,x_v,y_t,y_v = train_test_split(train['tweet'],train['class'],test_size=0.2,random_state=234)

test_tokenized = test_w2v.apply(lambda r: w2v_tokenize_text(r['tweet']), axis=1).values
train_tokenized = train_w2v.apply(lambda r: w2v_tokenize_text(r['tweet']), axis=1).values

X_train_word_average = word_averaging_list(wv,train_tokenized)
X_test_word_average = word_averaging_list(wv,test_tokenized)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `syn0norm` (Attribute will be removed in 4.0.0, use self.wv.vectors_norm instead).
  
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [38]:
### out of time #1

real_world_test_sample = real_world_test.iloc[400:500]
real_world_tokenized = real_world_test_sample.apply(lambda r: w2v_tokenize_text(r['tweet']), axis=1).values

out_of_time_test = word_averaging_list(wv,real_world_tokenized)

### out of time #2

real_world_test_sample_racist = real_world_test_racist.iloc[200:400]
real_world_tokenized_racist = real_world_test_sample_racist.apply(lambda r: w2v_tokenize_text(r['tweet']), axis=1).values

out_of_time_test_racist = word_averaging_list(wv,real_world_tokenized_racist)


### out of time #3

pok_sample = pok.iloc[200:400]
pok_tokenized = pok_sample.apply(lambda r: w2v_tokenize_text(r['tweet']), axis=1).values

pok_out_of_time = word_averaging_list(wv,pok_tokenized)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `syn0norm` (Attribute will be removed in 4.0.0, use self.wv.vectors_norm instead).
  
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [0]:
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import f1_score, classification_report, roc_auc_score, confusion_matrix, accuracy_score

In [0]:
def model_training(clf, x_t, y_t, x_v=None , y_v=None ,task='binary:logistic'):
    clf.fit(x_t,y_t)
    print('training accuracy', clf.score(x_t,y_t))
    
    if task=='binary:logistic':
      print('validation accuracy', clf.score(x_v,y_v))
      print('validation f1_score',f1_score(clf.predict(x_v),y_v))
      print('validation roc_auc score',roc_auc_score(y_v,clf.predict_proba(x_v)[::,-1]))
      print('confusion matrix \n',confusion_matrix(y_v, clf.predict(x_v)))
    
    if task=='reg:linear':
      print('validation r2_score', clf.score(x_v,y_v))
      print('validation MSE',mean_squared_error(clf.predict(x_v),y_v))

            
    return clf

In [41]:
%%time
xgb_w2v = XGBClassifier(n_estimators=500, max_depth=5,learning_rate=0.1,scale_pos_weight=1.4266790777602751)
model_training(xgb_w2v,X_train_word_average,train_w2v['class'],X_test_word_average,test_w2v['class'])

training accuracy 0.9921845336033414
validation accuracy 0.8696532523411794
validation f1_score 0.8958122597612785
validation roc_auc score 0.9436400805926967
confusion matrix 
 [[2444  488]
 [ 542 4428]]
CPU times: user 10min 15s, sys: 594 ms, total: 10min 16s
Wall time: 10min 17s


In [42]:
print('validation accuracy', xgb_w2v.score(X_test_word_average,test_w2v['class']))
print('validation f1_score',f1_score(xgb_w2v.predict(X_test_word_average),test_w2v['class']))

validation accuracy 0.8696532523411794
validation f1_score 0.8958122597612785


In [44]:
import joblib
#save model
joblib.dump(xgb_w2v, 'xgb_final_model.pkl') 

# #load saved model
# xgb = joblib.load(filename)

['xgb_final_model.pkl']

In [45]:
confusion_matrix(test_w2v['class'],xgb_w2v.predict(X_test_word_average))

array([[2444,  488],
       [ 542, 4428]])

In [46]:
tuned_pred = (xgb_w2v.predict_proba(X_test_word_average)[::,-1]>0.3).astype(int)
confusion_matrix(test_w2v['class'],tuned_pred)

array([[2097,  835],
       [ 311, 4659]])

In [47]:
f1_score(test_w2v['class'],tuned_pred)

0.8904816513761468

In [48]:
test_w2v['class'].value_counts()

1    4970
0    2932
Name: class, dtype: int64

In [0]:
pd.set_option('display.max_colwidth', -1)

#### Out of time #1 -  Trump Tweets

In [50]:
real_world_test_original = pd.read_csv('/content/drive/My Drive/Hate_Speech_Detection_git/real_world_test/realDonaldTrump_tweets_extended.csv')

check_df = real_world_test_original.iloc[400:500]
check_df['processed_tweet'] = real_world_test.iloc[400:500]['tweet']
check_df['preds'] = xgb_w2v.predict(out_of_time_test)
check_df['hate_probab'] = xgb_w2v.predict_proba(out_of_time_test)[::,-1]

# false_index = check_df[check_df['class']==xgb_w2v.predict(X_test_word_average)]['ID'].unique()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

In [51]:
check_df[check_df['hate_probab']>0.5]

,id,created_at,text,processed_tweet,preds,hate_probab
421,1182290380542185472,2019-10-10 13:42:47,b'Why isn\xe2\x80\x99t the IG investigating his so-called Whistleblower? All bad info!',isnxexxt ig investigating socalled whistleblower bad info,1,0.536151
444,1181940394604535809,2019-10-09 14:32:04,"b'....DACA stand, with all of its negative legal implications, the Republicans and Democrats will have a DEAL to let them stay in our Country, in very short order. It would actually benefit DACA, and be done the right way!'",daca stand negative legal implication republican democrat deal let stay country short order would actually benefit daca done right way,1,0.565205
445,1181940389990805504,2019-10-09 14:32:03,"b'President Obama said that he did not have the right to sign DACA, that it will never hold up in court. He signed it anyway! If the Supreme Court upholds DACA, it gives the President extraordinary powers, far greater than ever thought. If they do what is right and do not let......'",president obama said right sign daca never hold court signed anyway supreme court upholds daca give president extraordinary power far greater ever thought right let,1,0.564453
447,1181935014709202947,2019-10-09 14:10:41,b'Crooked Hillary should try it again! https://t.co/UjfIpZp1FA',crooked hillary try,1,0.802087
450,1181933850991235073,2019-10-09 14:06:04,"b'Why doesn\xe2\x80\x99t the ICIG do something about this Scam? He should have never let it start, a Dem Hoax! https://t.co/UCbObppWbA'",doesnxexxt icig something scam never let start dem hoax,1,0.761173
453,1181932207767789571,2019-10-09 13:59:32,"b'So true Don, and this was a big lie. Look what happened to Friends of Trump, for less. Can\xe2\x80\x99t let this pass! https://t.co/u2To514DNq'",true big lie look happened friend trump le canxexxt let pas,1,0.648571
459,1181929745518403585,2019-10-09 13:49:45,"b'Great support from GOP in fighting the Radical Left, Do Nothing Democrats! https://t.co/qjskHYkQpD'",great support gop fighting radical left nothing democrat,1,0.668431
469,1181891356274413570,2019-10-09 11:17:12,"b'So make all Member Countries pay, not just the United States! https://t.co/IVbE4MqBVl'",make member country pay united state,1,0.520830
474,1181883934776401920,2019-10-09 10:47:43,"b'A Total Scam by the Do Nothing Democrats. For the good of the Country, this Wirch Hunt should end now! https://t.co/G2nPapozye'",total scam nothing democrat good country wirch hunt end,1,0.514024
482,1181750932444520448,2019-10-09 01:59:12,"b'RT @senatemajldr: The USMCA would create 176,000 jobs for American workers and generate $68 billion in wealth for America. House Democrats\xe2\x80\xa6'",usmca would create job american worker generate billion wealth america house democratsxexxa,1,0.552511


#### Out of Time #2 -  Racist Tweets
P.S. Not all of them are racist actually, we need to check it manually

In [52]:
real_world_test_original_racist = pd.read_csv('/content/drive/My Drive/Hate_Speech_Detection_git/real_world_test/racist_scarped_extended.csv', names=['timestamp','text'])

check_df_2 = real_world_test_original_racist.iloc[200:400]
check_df_2['processed_tweet'] = real_world_test_racist.iloc[200:400]['tweet']
check_df_2['preds'] = xgb_w2v.predict(out_of_time_test_racist)
check_df_2['hate_probab'] = xgb_w2v.predict_proba(out_of_time_test_racist)[::,-1]

# false_index = check_df[check_df['class']==xgb_w2v.predict(X_test_word_average)]['ID'].unique()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

In [53]:
check_df_2[check_df_2['hate_probab']>0.4]

,timestamp,text,processed_tweet,preds,hate_probab
207,2019-10-24 09:27:26,b'\xe2\x81\xa6@TheJire\xe2\x81\xa9 and \xe2\x81\xa6@neccowafers\xe2\x81\xa9 harass gay black men with homophobic and racist slurs #gaymers #blacklgbt #black #racist https://t.co/8oItTxtyFH',harass gay black men homophobic racist slur,1,0.926347
209,2019-10-24 09:25:12,"b'@MiriamElman @DeborahProudJew @SkepticReview89 @transform6789 @DavidHirsh @bariweiss @fathomjournal AntiZionism = antiSemitism\n\nThe resistance to a Jewish State is as old as the idea itself, and it\xe2\x80\x99s not based on \xe2\x80\x98Israel\xe2\x80\x99s behaviour\xe2\x80\x99, as some on the Left like to portray it. (See above) The resistance to Israel is #racist.'",antizionism antisemitismnnthe resistance jewish state old idea itxexxs based xexxisraelxexxs behaviourxexx left like portray see resistance israel,0,0.474567
210,2019-10-24 09:21:40,b'I really doubt in this whole world that white person of anywhere can ever get over that stupid fact of thinking they know a Shit and are shit ... when truly they are this worlds biggest Fools Nd Bullshit ... #WhitePrivilege #WhiteSupremacist #Cracker #racist https://t.co/U2SK4X3Bnx',really doubt whole world white person anywhere ever get stupid fact thinking know shit shit truly world biggest fool nd bullshit,0,0.480192
216,2019-10-24 08:46:55,"b'RT @ClimateCritic18: #auspol\n\nYou are labeled #racist, privileged, white supremacist, #bigoted, #ableist, #ignorant and a nazi just for the\xe2\x80\xa6'",labeled privileged white supremacist nazi thexexxa,1,0.688225
217,2019-10-24 08:43:43,"b'RT @ClimateCritic18: #auspol\n\nYou are labeled #racist, privileged, white supremacist, #bigoted, #ableist, #ignorant and a nazi just for the\xe2\x80\xa6'",labeled privileged white supremacist nazi thexexxa,1,0.688225
...,...,...,...,...,...
358,2019-10-24 02:35:19,b'@cgrapski @AfricaMustWake @Nastyness61 #racist prosecutors employed by a predominantly white judiciary with #ImpeachedCops and equally racist and unqualified judges like #JudgeDonHLester @flcourts who subvert equal rights and protection under color of law #MandatedPOLY',prosecutor employed predominantly white judiciary equally racist unqualified judge like subvert equal right protection color law,1,0.732867
370,2019-10-24 02:02:40,"b""Question.... Isn't not letting someone say nigger based on their skin color kinda racist in itself. It's a paradox. #racist #nword""",question isnt letting someone say nigger based skin color kinda racist paradox,1,0.601461
371,2019-10-24 02:02:30,"b'@CornelWest @RefuseFascism YOU are the #Fascist and 100% ignorant\n\nIt was the founders of the #KKK (#democrats) doing the lynching \n\nAlso, #lynching\'s happened to white men by the 1000s, you dont own the term ""lynching""\n\nDemocrats are the #racist\n\nExhibit A: #BLACKFACE https://t.co/NR0awdyGr7'",ignorantnnit founder lynching nnalso happened white men dont term lynchingnndemocrats,1,0.761431
379,2019-10-24 01:57:39,"b'@Ilhan Thank you for showing your true colors. You try to divide and destroy our country, make racist comments, put us down, insult us, lie about us, lie about our President. #WalkAway #racist #Hypocrite #DemocratsAreCorrupt #IStandWithPresidentTrump'",thank showing true color try divide destroy country make racist comment put u insult u lie u lie president,1,0.795588


#### Out of time #3 - POK

In [54]:
pok_original = pd.read_csv('/content/drive/My Drive/Hate_Speech_Detection_git/real_world_test/pok.csv', names=['timestamp','text'])
pok_original = pok_original.drop_duplicates(subset='text')

check_df_3 = pok_original.iloc[200:400]
check_df_3['processed_tweet'] = pok.iloc[200:400]['tweet']
check_df_3['preds'] = xgb_w2v.predict(pok_out_of_time)
check_df_3['hate_probab'] = xgb_w2v.predict_proba(pok_out_of_time)[::,-1]

# false_index = check_df[check_df['class']==xgb_w2v.predict(X_test_word_average)]['ID'].unique()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/s

In [55]:
check_df_3[check_df_3['hate_probab']>0.4]

,timestamp,text,processed_tweet,preds,hate_probab
405,2019-10-23 15:58:58,b'@rehankayanii Whole world knows #Who_is_Modi And What he can do. Now Kashmir is gone from your hands Now its time to take our #Pok Back.',whole world know kashmir gone hand time take back,0,0.471642
472,2019-10-23 14:20:20,"b'RT @Je_suis_perdu_7: @Rabipirzada There is nothing like IOK but yes #POK exists where coward #PakistanArmy kidnap, kill and rape people.\n\n#\xe2\x80\xa6'",nothing like iok yes exists coward kidnap kill rape,1,0.819460
490,2019-10-23 13:59:36,b'As per the libtards only #Muslims are humans and only they have #HumanRights all others are kafirs hence rights dont matter.. \n#PoK \n#attackonfreedom',per libtards human others kafir hence right dont matter,0,0.411190
492,2019-10-23 13:58:30,b'Dear @Ilhan they have been asked to prove their citizenship because they are living illegally in india.... you are too blind to see what is happing in #POK and #Baluchistan \n\nDid you ask this question to yourself why all the terrorists attack has been done by Muslims ??? https://t.co/BFKR7G608d',dear asked prove citizenship living illegally india blind see happing nndid ask question terrorist attack done muslim,1,0.612894
510,2019-10-23 13:44:26,"b'RT @Drg_nn: @DextrousNinja https://t.co/5CV3cfS9Of\nSomali Muslim Rep #IlhanOmar has lied, cheated &amp; played sympathy card to get her US citi\xe2\x80\xa6'",muslim rep lied cheated amp played sympathy card get u citixexxa,1,0.683725
552,2019-10-23 13:19:50,b'RT @BasudevOfficial: #ImranKhan declared #Emergence at #POK by scared of there sound gathered for freedom from #Pakistan .\n\nBy showing this\xe2\x80\xa6',declared scared sound gathered freedom nnby showing thisxexxa,0,0.465755
585,2019-10-23 12:47:25,"b'@Imamofpeace @IlhanMN What she have to say on butchering of #Kurds and Muslims in Middle East by Islamic fundamentalists. She ignore ethnic cleansing of #ahmadias, rape &amp; conversion of minority girls by #pakistani Muslims in #blochistan &amp; #POK . She should be ashamed of herself. We don\xe2\x80\x99t give a damn !'",say butchering muslim middle east islamic fundamentalist ignore ethnic cleansing rape amp conversion minority girl muslim amp ashamed donxexxt give damn,1,0.913208
587,2019-10-23 12:45:39,b'#IndianArmedForces must liberate #PoK tonight and give freedom to kashmiris living there from the atrocities of terrorist #PakistanArmy . 26 October 1947 is blackday for the people living in #PoK. #PoK is under illegal occupation of #PakistanArmy. #loc',must liberate tonight give freedom kashmiri living atrocity terrorist october blackday people living illegal occupation,1,0.750307
621,2019-10-23 12:05:20,"b""@arifaajakia Holy crap.. That's a United Nations car.. It is the @UN delegation.... People of #POK are protesting in front of them asking #pakistan to leave #kashmir. Hahaha \xf0\x9f\xa4\xa3\xf0\x9f\x98\x82\xf0\x9f\x99\x88\xf0\x9f\x99\x89\xf0\x9f\x99\x8a #Shameonpakistan #Kashmir #Kashmiris""",holy crap thats united nation car delegation people protesting front asking leave hahaha xfxfxaxaxfxfxxxfxfxxxfxfxxxfxfxxa,0,0.446734
636,2019-10-23 11:46:21,b'@Ilhan Have ur say on killing in #PoK by #PakistanArmy since last 2days?\nY ur quite on that?',ur say killing since last daysny ur quite,1,0.642303
